In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import neurokit2 as nk
import pyedflib
import pandas as pd
import os

## Data

In [3]:
# Directory of EDF files
edf_directory = '../datasetsPart2/valu3s/vitaport/'

In [4]:
# Directory of CSV files
csv_directory = '../datasets_2/valu3s/vitaport/filtered_signals/'

In [9]:
# Duration of each interval in seconds
interval_duration = 120 

# Initialize an empty DataFrame to store all features
feature_dfs = []

# Sampling Rate
sampling_rate = 256

## Process Freq Domain ECG

In [7]:
# Function extract featrures from ECG
def extract_hrv_features(signal, sampling_rate, interval_duration):
    # number of intrevals
    interval_samples = int(interval_duration * sampling_rate)
    features = [] 
    
    # loop to extract features for each intreval 
    for start in range(0, len(signal), interval_samples):
        end = start + interval_samples
        if end > len(signal):
            break
        
        interval_signal = signal[start:end]
        if len(interval_signal) < interval_samples:
            continue
        
        try:
            # extract features 
            signals, info = nk.ecg_process(interval_signal, sampling_rate=sampling_rate)
            hrv_features = nk.hrv_frequency(info['ECG_R_Peaks'], sampling_rate=sampling_rate, normalize=True)
            hrv_features['Interval_Start'] = start / sampling_rate
            hrv_features['Interval_End'] = end / sampling_rate

            # add extrated features to the DataFrame
            features.append(hrv_features)
        except Exception as e:
            print(f"Error processing interval {start}-{end}: {e}")
            continue

    return features

# Loop through all .edf files in the directory
for filename in os.listdir(edf_directory):
    if filename.endswith('.edf'):
        file_path = os.path.join(edf_directory, filename)
        
        # Load the .edf file
        f = pyedflib.EdfReader(file_path)
        
        # Get signal labels and find ECG signal index
        signal_labels = f.getSignalLabels()
        if 'ECG' in signal_labels:
            ecg_signal_index = signal_labels.index('ECG')
            ecg_signal = f.readSignal(ecg_signal_index)
            sampling_rate = f.getSampleFrequency(ecg_signal_index)
        else:
            print(f"No ECG signal found in {filename}")
            continue
        
        # Close the EDF file
        f.close()

        # duration of the ECG signal in seconds
        ecg_duration = len(ecg_signal) / sampling_rate
        print(f"ECG duration for {filename}: {ecg_duration} seconds")
        
        # Extract HRV features for each interval
        interval_features = extract_hrv_features(ecg_signal, sampling_rate, interval_duration)

        # number of intervals
        num_intervals = len(interval_features)
        print(f"Number of intervals for {filename}: {num_intervals}")
        
        if interval_features:
            # Convert list of feature dictionaries to DataFrame and add filename
            interval_features_df = pd.concat([pd.DataFrame(features) for features in interval_features])
            interval_features_df['Filename'] = filename
        
            # Append the features DataFrame to the list
            feature_dfs.append(interval_features_df)
        else:
            print(f"No valid intervals found in {filename}")

ECG duration for fp01_1.edf: 3970.0 seconds
Number of intervals for fp01_1.edf: 13
ECG duration for fp01_2.edf: 4020.0 seconds
Number of intervals for fp01_2.edf: 13
ECG duration for fp01_3.edf: 3770.0 seconds
Number of intervals for fp01_3.edf: 12
ECG duration for fp01_4.edf: 3920.0 seconds
Number of intervals for fp01_4.edf: 13
ECG duration for fp02_1.edf: 3840.0 seconds
Number of intervals for fp02_1.edf: 12
ECG duration for fp02_2.edf: 3810.0 seconds
Number of intervals for fp02_2.edf: 12
ECG duration for fp02_3.edf: 3780.0 seconds
Number of intervals for fp02_3.edf: 12
ECG duration for fp02_4.edf: 3880.0 seconds
Number of intervals for fp02_4.edf: 12
ECG duration for fp03_2.edf: 4030.0 seconds
Number of intervals for fp03_2.edf: 13
ECG duration for fp03_3.edf: 3940.0 seconds
Number of intervals for fp03_3.edf: 13
ECG duration for fp03_4.edf: 4260.0 seconds
Number of intervals for fp03_4.edf: 14
ECG duration for fp04_1.edf: 4070.0 seconds
Number of intervals for fp04_1.edf: 13
ECG 

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppDat

Error processing interval 76800-153600: integer division or modulo by zero
Error processing interval 153600-230400: integer division or modulo by zero
Error processing interval 230400-307200: integer division or modulo by zero
Error processing interval 307200-384000: integer division or modulo by zero
Error processing interval 384000-460800: integer division or modulo by zero
Error processing interval 460800-537600: integer division or modulo by zero
Error processing interval 537600-614400: integer division or modulo by zero
Error processing interval 614400-691200: integer division or modulo by zero
Error processing interval 691200-768000: integer division or modulo by zero
Error processing interval 768000-844800: integer division or modulo by zero
Error processing interval 844800-921600: integer division or modulo by zero
Error processing interval 921600-998400: integer division or modulo by zero
Number of intervals for fp13_1.edf: 1
ECG duration for fp13_2.edf: 4050.0 seconds


c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppDat

Error processing interval 76800-153600: cannot convert float NaN to integer
Error processing interval 153600-230400: integer division or modulo by zero
Error processing interval 230400-307200: integer division or modulo by zero
Error processing interval 307200-384000: integer division or modulo by zero
Error processing interval 384000-460800: integer division or modulo by zero
Error processing interval 460800-537600: integer division or modulo by zero
Error processing interval 537600-614400: integer division or modulo by zero
Error processing interval 614400-691200: integer division or modulo by zero
Error processing interval 691200-768000: integer division or modulo by zero
Error processing interval 768000-844800: integer division or modulo by zero
Error processing interval 844800-921600: integer division or modulo by zero
Error processing interval 921600-998400: integer division or modulo by zero
Number of intervals for fp13_2.edf: 1
ECG duration for fp13_3.edf: 3690.0 seconds
Error 

...

Error processing interval 460800-537600: integer division or modulo by zero
Error processing interval 537600-614400: integer division or modulo by zero
Error processing interval 614400-691200: cannot convert float NaN to integer
Error processing interval 691200-768000: cannot convert float NaN to integer
Error processing interval 768000-844800: cannot convert float NaN to integer
Error processing interval 844800-921600: cannot convert float NaN to integer
Number of intervals for fp13_3.edf: 0
No valid intervals found in fp13_3.edf
ECG duration for fp13_4.edf: 3710.0 seconds
Error processing interval 0-76800: integer division or modulo by zero
Error processing interval 76800-153600: integer division or modulo by zero
Error processing interval 153600-230400: integer division or modulo by zero
Error processing interval 230400-307200: integer division or modulo by zero
Error processing interval 307200-384000: integer division or modulo by zero
Error processing interval 384000-460800: canno

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppDat

Error processing interval 768000-844800: integer division or modulo by zero
Error processing interval 844800-921600: integer division or modulo by zero
Number of intervals for fp13_4.edf: 0
No valid intervals found in fp13_4.edf
ECG duration for fp14_1.edf: 4010.0 seconds
Number of intervals for fp14_1.edf: 13
ECG duration for fp14_2.edf: 4060.0 seconds
Number of intervals for fp14_2.edf: 13
ECG duration for fp14_3.edf: 4340.0 seconds
Number of intervals for fp14_3.edf: 14
ECG duration for fp14_4.edf: 4180.0 seconds
Number of intervals for fp14_4.edf: 13
ECG duration for fp15_1.edf: 3840.0 seconds
Number of intervals for fp15_1.edf: 12
ECG duration for fp15_2.edf: 3840.0 seconds
Number of intervals for fp15_2.edf: 12
ECG duration for fp15_3.edf: 4160.0 seconds
Number of intervals for fp15_3.edf: 13
ECG duration for fp15_4.edf: 4010.0 seconds
Number of intervals for fp15_4.edf: 13
ECG duration for fp16_1.edf: 3930.0 seconds
Number of intervals for fp16_1.edf: 13
ECG duration for fp16_2.

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(


Error processing interval 384000-460800: index 11 is out of bounds for axis 0 with size 9
Error processing interval 460800-537600: index 10 is out of bounds for axis 0 with size 9
Error processing interval 537600-614400: index 11 is out of bounds for axis 0 with size 9
Error processing interval 614400-691200: index 11 is out of bounds for axis 0 with size 9
Error processing interval 691200-768000: index 10 is out of bounds for axis 0 with size 9
Number of intervals for fp19_1.edf: 2
ECG duration for fp19_2.edf: 3980.0 seconds
Number of intervals for fp19_2.edf: 13
ECG duration for fp19_3.edf: 3370.0 seconds
Number of intervals for fp19_3.edf: 11
ECG duration for fp20_1.edf: 3850.0 seconds
Number of intervals for fp20_1.edf: 12
ECG duration for fp20_2.edf: 3890.0 seconds
Number of intervals for fp20_2.edf: 12
ECG duration for fp20_3.edf: 3160.0 seconds
Number of intervals for fp20_3.edf: 10
ECG duration for fp20_4.edf: 3730.0 seconds
Number of intervals for fp20_4.edf: 12


## Save Features Dataset EDF

In [8]:
# Concatenate all feature DataFrames into one DataFrame
all_features_df = pd.concat(feature_dfs, ignore_index=True)

# Save the features DataFrame to a CSV file
all_features_df.to_csv('hrv_freq_domain_5_min.csv', index=False)

print("Saved features on 'hrv_freq_domain_5_min.csv'.")

Saved features on 'hrv_freq_domain_5_min.csv'.


## Process Freq Domain CSV

In [10]:
# Function extract featrures from ECG
def extract_hrv_features(signal, sampling_rate, interval_duration):
    # number of intrevals
    interval_samples = int(interval_duration * sampling_rate)
    features = [] 
    
    # loop to extract features for each intreval 
    for start in range(0, len(signal), interval_samples):
        end = start + interval_samples
        if end > len(signal):
            break
        
        interval_signal = signal[start:end]
        if len(interval_signal) < interval_samples:
            continue
        
        try:
            # extract features 
            signals, info = nk.ecg_process(interval_signal, sampling_rate=sampling_rate)
            hrv_features = nk.hrv_frequency(info['ECG_R_Peaks'], sampling_rate=sampling_rate, normalize=True)
            hrv_features['Interval_Start'] = start / sampling_rate
            hrv_features['Interval_End'] = end / sampling_rate

            # add extrated features to the DataFrame
            features.append(hrv_features)
        except Exception as e:
            print(f"Error processing interval {start}-{end}: {e}")
            continue

    return features

# Loop all edf files 
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_directory, filename)
        
        # Load the CSV file 
        try:
            # Read CSV 
            df = pd.read_csv(file_path, header=None)
            ecg_signal = df[0].values 
        except Exception as e:
            print(f"Error loading {filename}: {e}")
            continue

        # duration of the ECG signal in seconds
        ecg_duration = len(ecg_signal) / sampling_rate
        print(f"ECG duration for {filename}: {ecg_duration} seconds")
        
        # Extract HRV features for each interval
        interval_features = extract_hrv_features(ecg_signal, sampling_rate, interval_duration)

        # Number of intervals
        num_intervals = len(interval_features)
        print(f"Number of intervals for {filename}: {num_intervals}")
        
        if interval_features:
            # Convert list of feature dictionaries to DataFrame and add filename
            interval_features_df = pd.concat([pd.DataFrame(features) for features in interval_features])
            interval_features_df['Filename'] = filename
        
            # Append the features DataFrame to the list
            feature_dfs.append(interval_features_df)
        else:
            print(f"No valid intervals found in {filename}")

ECG duration for fp01_1.csv: 3694.578125 seconds
Number of intervals for fp01_1.csv: 30
ECG duration for fp01_2.csv: 3576.703125 seconds
Number of intervals for fp01_2.csv: 29
ECG duration for fp01_3.csv: 3574.078125 seconds
Number of intervals for fp01_3.csv: 29
ECG duration for fp01_4.csv: 3734.52734375 seconds
Number of intervals for fp01_4.csv: 31
ECG duration for fp02_1.csv: 3721.453125 seconds
Number of intervals for fp02_1.csv: 31
ECG duration for fp02_2.csv: 3691.203125 seconds
Number of intervals for fp02_2.csv: 30
ECG duration for fp02_3.csv: 3575.078125 seconds
Number of intervals for fp02_3.csv: 29
ECG duration for fp02_4.csv: 3668.953125 seconds
Number of intervals for fp02_4.csv: 30
ECG duration for fp03_2.csv: 3990.5078125 seconds
Number of intervals for fp03_2.csv: 33
ECG duration for fp03_3.csv: 3580.953125 seconds
Number of intervals for fp03_3.csv: 29
ECG duration for fp03_4.csv: 3573.328125 seconds
Number of intervals for fp03_4.csv: 29
ECG duration for fp04_1.csv: 

c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Loca

ECG duration for fp13_2.csv: 3573.578125 seconds
Error processing interval 0-30720: index 9 is out of bounds for axis 0 with size 9


c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Loca

Error processing interval 30720-61440: cannot convert float NaN to integer
Error processing interval 61440-92160: integer division or modulo by zero
Error processing interval 92160-122880: integer division or modulo by zero
Error processing interval 122880-153600: integer division or modulo by zero
Error processing interval 153600-184320: integer division or modulo by zero
Error processing interval 184320-215040: integer division or modulo by zero
Error processing interval 215040-245760: integer division or modulo by zero
Error processing interval 245760-276480: integer division or modulo by zero
Error processing interval 276480-307200: integer division or modulo by zero
Error processing interval 307200-337920: index 0 is out of bounds for axis 0 with size 0
Error processing interval 337920-368640: integer division or modulo by zero
Error processing interval 368640-399360: integer division or modulo by zero
Error processing interval 399360-430080: integer division or modulo by zero
Err

...

Error processing interval 430080-460800: index 10 is out of bounds for axis 0 with size 9
Error processing interval 460800-491520: cannot convert float NaN to integer
Error processing interval 491520-522240: cannot convert float NaN to integer
Error processing interval 522240-552960: cannot convert float NaN to integer
Error processing interval 552960-583680: cannot convert float NaN to integer
Error processing interval 583680-614400: integer division or modulo by zero
Error processing interval 614400-645120: cannot convert float NaN to integer
Error processing interval 645120-675840: cannot convert float NaN to integer


c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Prog

Error processing interval 675840-706560: index 9 is out of bounds for axis 0 with size 9
Error processing interval 706560-737280: cannot convert float NaN to integer
Error processing interval 737280-768000: index 9 is out of bounds for axis 0 with size 9
Error processing interval 768000-798720: cannot convert float NaN to integer


c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
c:\Users\duart\AppData\Local\Programs\Python\Python39\lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(


Number of intervals for fp19_1.csv: 5
ECG duration for fp19_2.csv: 3582.71484375 seconds
Number of intervals for fp19_2.csv: 29
ECG duration for fp19_3.csv: 3015.7578125 seconds
Number of intervals for fp19_3.csv: 25
ECG duration for fp20_1.csv: 3712.078125 seconds
Number of intervals for fp20_1.csv: 30
ECG duration for fp20_2.csv: 3577.953125 seconds
Number of intervals for fp20_2.csv: 29
ECG duration for fp20_3.csv: 3045.82421875 seconds
Number of intervals for fp20_3.csv: 25
ECG duration for fp20_4.csv: 3628.453125 seconds
Number of intervals for fp20_4.csv: 30


## Save Features Dataset EDF

In [11]:
# Concatenate all feature DataFrames into one DataFrame
all_features_df = pd.concat(feature_dfs, ignore_index=True)

# Save the features DataFrame to a CSV file
all_features_df.to_csv('hrv_freq_domain_2_min_filtered.csv', index=False)

print("Saved features on 'hrv_freq_domain_2_min_filtered.csv'.")

Saved features on 'hrv_freq_domain_2_min_filtered.csv'.
